In [1]:
import sys
from refine import *
import project
import mdtraj as mdt

import multiprocessing
from multiprocessing import Process, Queue, Lock
from queue import Empty
# multiprocessing.set_start_method("fork")

import io

%matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style

style.use("fivethirtyeight")

Using matplotlib backend: TkAgg


In [2]:
project.setup()

In [3]:
import pickle
with open(project.output_path / "session_5.session", "rb") as input_file:
    session = pickle.load(input_file)
print(session)

{'coords0': array([[ 15.101,  25.279, -11.672],
       [ 14.584,  26.181, -11.791],
       [ 15.683,  25.386, -10.815],
       ...,
       [ -8.473,  24.265,  10.977],
       [ -6.969,  23.383,  11.4  ],
       [ -7.024,  22.255,  14.01 ]]), 'coords1': array([[41.989, 28.525,  1.215],
       [41.413, 27.839,  1.745],
       [41.619, 28.594,  0.242],
       ...,
       [ 8.107, 33.273, -3.852],
       [ 8.463, 34.076, -4.244],
       [10.602, 32.38 , -1.59 ]]), 'initial_position': [array([0, 0, 0]), array([0.99144486, 0.        , 0.        , 0.13052619]), array([0.3, 0.1, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ])], 'result': {'positions': [[array([ 0.06258083,  1.44673025, -1.04684348]), array([ 0.9802652 , -0.10393313,  0.00633735,  0.13343592]), array([12.19425817,  2.26452367, 18.56486005,  2.96360493,  0.2255941 ,
       -5.89117046, 15.26653641,  6.9364498 , -9.71182438,  4.99610138])], [array([ 0.22653881,  2.24088776, -0.34921512]), array([ 0.97718794,  0.01238916, -0.00615622,  0

In [3]:
def confined_gradient_descent(
        obj_q, decrement=0.9, termination="growth",
        absolute_bound=float("inf"), relative_bounds=(0.01, 7.0), etol=1, max_iter=100, return_traj=False):
    """
    Performs gradient descent of a system with respect to a special confinement.

    @param nmw: system to optimize.
    @type nmw: NMSpaceWrapper
    @param decrement: fold step when choosing optimal step.
    @type decrement: float
    @param termination: termination condition.
    @type termination: str
    @param absolute_bound: maximum rmsd between inital state and any intermediate state.
    @type absolute_bound: float
    @param relative_bounds: minimum and maximum rmsd between actual intermediate state and the next one.
    @type relative_bounds: tuple
    @param etol: terminates when |E(i+1) - E(i)| < etol
    @type etol: float
    @param max_iter: maximum number of iterations
    @type max_iter: int
    @param return_traj: if true all intermediate states, energies and forces are returned.
        Otherwise, the function returns only final record.
    @type return_traj: bool
    @return: dictionary containing all the results.
        "states" - list of all states along optimization path.
        "energies" - list of all energies along optimization path.
        "forces" - list of all forces along optimization path.
        If return_traj is false returns only last record.
    @rtype: dict
    """
    
    path = str((project.data_path / "may_complex") / "1dfj.pdb")
    print(path)
    create_system(path, tmp_file=str(project.output_path / "tmp_system.pdb"))
    t0 = time.time()
    drs = DRSystem(str(project.output_path / "tmp_system.pdb"), 'amber14/protein.ff14SB.xml', refine="chain A", static="chain B")
    print("construction of a system:", -t0 + time.time(), "sec")
    t0 = time.time()
    nmw1 = NMSpaceWrapper(drs, n_modes=10)
    print("INIT ENERGY:", nmw1.get_energy())
    
    m = 0
    
    while m != 10:
    
        natoms = len(nmw1.get_system_position())
        weights = np.ones((natoms,))
        
        # random noise
        preset_rmsd = .3

        # mu = a * mu_0
        # preset_rmsd = |a| (np.dot(mu_0, mu_0) / natoms) ** 0.5 => 
        # => a = (preset_rmsd ** 2 * natoms / np.dot(mu_0, mu_0)) ** 0.5

        mu = np.random.normal(0, 1, 10)
        mu = (preset_rmsd ** 2 * natoms / np.dot(mu, mu)) ** 0.5 * mu
        nmw1.set_position(np.zeros(10,))
        x1 = nmw1.get_system_position().copy()
        nmw1.set_position(mu)
        x2 = nmw1.get_system_position().copy()
        print("rmsd init:", rmsd(x1, x2, weights))
        print("RANDOM ENERGY:", nmw1.get_energy())
        pdy.writePDB(str(project.output_path / "shifted_6.0.pdb"), drs._refine_prot)

        nmw2 = NMSpaceWrapper(drs, n_modes=10)
        m = len(nmw2.get_eigenvalues())
    
    nmw2.set_position(np.zeros((10,)))
    nmw = nmw2
    
    # number of modes
    m = len(nmw.get_eigenvalues())
    # number of atoms
    n = len(nmw.get_system_position())
    # modes
    modes = nmw.get_modes()
    # special values
    ru_bound_value = n * relative_bounds[1] ** 2
    rl_bound_value = n * relative_bounds[0] ** 2
    a_bound_value = n * absolute_bound ** 2
    # initial state
    iteration_count = 0
    states = []
    energies = []
    forces = []
    energy_init = nmw.get_energy()
    position_init = nmw.get_position().copy()
    energies.append(energy_init)
    states.append(nmw.get_system_position().copy())

    # it is a weighted anti-gradient!
    anti_gradient = nmw.get_force().copy()
    forces.append(anti_gradient)
    
    # update data
    obj_q.put({
        "energies": energies,
        "states": states,
        "forces": forces,
    })
    
    # main cycle
    while True:
        print("\n(main) CYCLE START\n")

        # find a step
        # relative bound
        a = np.dot(anti_gradient, anti_gradient)
        upper_bound = (ru_bound_value / a) ** 0.5
        lower_bound = (rl_bound_value / a) ** 0.5
        # absolute bound


        # find optimal step
        dec = [1]
        eng = []
        ind = [0]

        nmw.set_position(dec[0] * upper_bound * anti_gradient + position_init)

        eng.append(nmw.get_energy())
        score = 0
        while True:
            dec.append(dec[-1] * decrement)
            ind.append(ind[-1] + 1)
            if dec[-1] * upper_bound <= lower_bound:
                break

            nmw.set_position(dec[-1] * upper_bound * anti_gradient + position_init)
            eng.append(nmw.get_energy())
            print("\t(line) energy:", eng[-1])
            # update data
            obj_q.put({
                "energies_line": eng,
            })
            
            # exit condition
            if energy_init > eng[-1] > eng[-2]:
                score += 1
            elif eng[-1] < eng[-2] or energy_init <= eng[-1]:
                score = 0
            if score >= 3:
                break

        # update state list
        j = np.argmin(eng)
        mu = dec[j] * upper_bound * anti_gradient

        # new initial states
        position_init = mu + position_init
        energy_init = eng[j]
        nmw.set_position(position_init)

        # state energy force
        states.append(nmw.get_system_position().copy())
        energies.append(eng[j])
        anti_gradient = nmw.get_force().copy()
        forces.append(anti_gradient)
        
        # print results
        print("(main) optimum:")
        print("       index:", j)
        print("       energy:", energies[-1])
        print("       force:", np.linalg.norm(forces[-1]))
        
        # adaptive upper bound
        ru_bound_value = min(np.dot(mu, mu) / decrement ** 3, n * relative_bounds[1] ** 2)
        print("       upper_bound:", (ru_bound_value / n) ** 0.5)
        
        # termination
        iteration_count += 1
        
        # update data
        obj_q.put({
            "energies": energies,
            "states": states,
            "forces": forces,
        })
        
        if termination == "growth":
            if energy_init < eng[j]:
                states.pop()
                energies.pop()
                forces.pop()
                break
        elif termination == "etol":
            if abs(eng[j] - energy_init) < etol:
                break
        else:
            raise ValueError(f"Wrong termination criterion: {termination}")
        if iteration_count >= max_iter:
            break
    if not return_traj:
        return {"states": states[-1:], "energies": energies[-1:]}
    else:
        return {"states": states, "energies": energies, "forces": forces}

## Model setup

In [4]:
path = str((project.data_path / "may_complex") / "1dfj.pdb")
print(path)
create_system(path, tmp_file=str(project.output_path / "tmp_system.pdb"))
t0 = time.time()
drs = DRSystem(str(project.output_path / "tmp_system.pdb"), 'amber14/protein.ff14SB.xml', refine="chain A", static="chain B")
print("construction of a system:", -t0 + time.time(), "sec")
t0 = time.time()
nmw1 = NMSpaceWrapper(drs, n_modes=10)
print("INIT ENERGY:", nmw1.get_energy())

@> 4416 atoms and 1 coordinate set(s) were parsed in 0.13s.


/home/semyon/PycharmProjects/DiplomaPython/data/may_complex/1dfj.pdb
write PDB(prody): 0.0381 sec
read PDB(openmm): 0.2993302345275879 sec


@> 8726 atoms and 1 coordinate set(s) were parsed in 0.06s.


add hydrogens and extra particles(openmm): 18.434768199920654 sec
write PDB(openmm): 0.09213900566101074 sec
1856 6870
construction of a system: 1.5710911750793457 sec


@> Hessian was built in 7.83s.
@> 10 modes were calculated in 20.03s.


INIT ENERGY: -29981.58122926692


In [5]:
nmw1.set_position(np.zeros(10,))

In [6]:
nmw2 = NMSpaceWrapper(drs, n_modes=10, cutoff=4)

@> Hessian was built in 0.80s.
@> 10 modes were calculated in 23.43s.


## Tests

In [8]:
def calc_center(m, weights=None):
    n = m.shape[0]
    m = m.reshape((n // 3, 3))
    if weights is None:
        return np.apply_along_axis(np.average, 0, m)
    else:
        return np.apply_along_axis(lambda x: np.average(x, weights=weights), 0, m)

In [26]:
modes = nmw2.get_modes()
for m in modes:
    print(np.linalg.norm(calc_center(m, weights=np.ones((len(m) // 3, )))))

1.521046477259055e-15
7.190312379372309e-16
9.999545806823713e-17
5.717198892408634e-16
4.3993788526774136e-16
2.8029245073475843e-16
3.765827804049881e-16
2.3579353032804193e-16
1.389263382343469e-16
7.940449149251252e-17


In [16]:
natoms = len(nmw2.get_system_position())
x = np.arange(0, 1, 0.2) * natoms ** 0.5
v = np.zeros((10,))
v[0] = 1
for xi in x:
    nmw2.set_position(xi * v)
    print("geom", pdy.calcCenter(drs._refine_prot))
nmw2.set_position(np.zeros((10,)))
print()
for xi in x:
    nmw2.set_position(xi * v)
    print("mass", pdy.calcCenter(drs._refine_prot, weights=drs._refine_prot.getMasses()))
nmw2.set_position(np.zeros((10,)))

geom [ 5.35085937 24.4358847   6.60488039]
geom [ 5.35085937 24.4358847   6.60488039]
geom [ 5.35085937 24.4358847   6.60488039]
geom [ 5.35085937 24.4358847   6.60488039]
geom [ 5.35085937 24.4358847   6.60488039]

mass [ 5.3253999  24.37219457  6.65973308]
mass [ 5.32417678 24.36898966  6.66014849]
mass [ 5.32295366 24.36578476  6.66056389]
mass [ 5.32173054 24.36257985  6.66097929]
mass [ 5.32050743 24.35937494  6.6613947 ]


### Testing triplet wise cross product orthogonality for normal modes

In [22]:
mode1 = nmw2.get_modes()[0]
mode2 = nmw2.get_modes()[1]
thn = mode1.shape[0]
mode1 = np.reshape(mode1, (thn // 3, 3))
mode2 = np.reshape(mode2, (thn // 3, 3))
cross_mode = []
for i in range(thn // 3):
    cross_mode.append(np.cross(mode1[i], mode2[i]))
cross_mode = np.array(cross_mode).reshape((thn,))
print(calc_center(cross_mode))
print(calc_center(nmw2.get_modes()[1]))
print(np.dot(cross_mode, mode1.reshape((thn,))))
print(np.dot(cross_mode, mode2.reshape((thn,))))

[-1.41288461e-05 -5.24650573e-07  8.68634828e-08]
[7.03460279e-16 1.65935273e-16 1.83761052e-16]
1.3552527156068805e-20
1.1519648082658485e-19


### Testing pairwise distance

In [17]:
# test modes
natoms = len(nmw2.get_system_position())
alpha = 1.5
x = np.arange(0, alpha, 0.1)
x = np.concatenate((x, np.flip(x), -x, np.flip(-x)))
coords = []
pos = np.random.uniform(-1, 1, 10)
pos = pos / np.linalg.norm(pos)
for g in x:
    nmw2.set_position(g * natoms ** 0.5 * pos)
    coords.append(nmw2.get_system_position())
tmp_file = project.output_path / "tmp_system.pdb"
output_file = project.output_path / "trajectory.pdb"
save_trajectory(drs, coords, tmp_file=tmp_file, output_file=output_file, log=sys.stdout)

Trajectory info:
	number of states: 60
	temporary file: /home/semyon/PycharmProjects/DiplomaPython/output/tmp_system.pdb
	ouput file: /home/semyon/PycharmProjects/DiplomaPython/output/trajectory.pdb
creating trajectory: 1.7%
creating trajectory: 10.0%
creating trajectory: 18.3%
creating trajectory: 26.7%
creating trajectory: 35.0%
creating trajectory: 43.3%
creating trajectory: 51.7%
creating trajectory: 60.0%
creating trajectory: 68.3%
creating trajectory: 76.7%
creating trajectory: 85.0%
creating trajectory: 93.3%
Saving trajectory...
Done!


In [ ]:
single_upper_bound = 1 #
modes = nmw1.get_modes()
max_values = []
for i in range(10):
    mode_xyz = modes[i].reshape((len(modes[i]) // 3, 3))
    max_values.append(single_upper_bound / np.max(np.apply_along_axis(np.linalg.norm, 1, mode_xyz)))
fig, ax = plt.subplots()
ax.set_xlabel("Mode number")
ax.set_ylabel("Maximum rmsd upper bound for each mode")
ax.set_xticks(np.arange(1, 11, 1))
ax.set_yticks(np.arange(0, 10, 0.4))
ax.bar(np.arange(1, 11, 1), max_values)

In [ ]:
pos = nmw1.get_system_position()
print(pos.shape)
pair_dist = []
for i in range(1, pos.shape[0]):
    pair_dist.append(np.linalg.norm(pos[i] - pos[i-1]))
print(np.max(pair_dist))

### Random test

In [ ]:
natoms = len(nmw1.get_system_position())
weights = np.ones((natoms,))

In [ ]:
# random noise
preset_rmsd = .7

# mu = a * mu_0
# preset_rmsd = |a| (np.dot(mu_0, mu_0) / natoms) ** 0.5 => 
# => a = (preset_rmsd ** 2 * natoms / np.dot(mu_0, mu_0)) ** 0.5

mu = np.random.normal(0, 1, 10)
mu = (preset_rmsd ** 2 * natoms / np.dot(mu, mu)) ** 0.5 * mu
nmw1.set_position(np.zeros(10,))
x1 = nmw1.get_system_position().copy()
nmw1.set_position(mu)
x2 = nmw1.get_system_position().copy()
print("RMSD INIT:", rmsd(x1, x2, weights))
print("RANDOM ENERGY:", nmw1.get_energy())
print("NM VECTOR:", mu)
pdy.writePDB(str(project.output_path / "shifted_6.0.pdb"), drs._refine_prot)

In [ ]:
nmw2 = NMSpaceWrapper(drs, n_modes=10)

In [ ]:
old_coords = nmw2.get_system_position().copy()

In [ ]:
nmw2.set_position(np.zeros((10,)))

### Test descent

In [7]:
# mutiprocessing
obj_q = Queue(maxsize=20)
full_list = []
# it is impossible to pickle drs system => can not be transfered between processes

# gradient descent process
uprocess = Process(target=confined_gradient_descent, args=(obj_q,),
                   kwargs={"relative_bounds": (0.001, 0.25),
                           "decrement": 0.9,
                           "max_iter": 100,
                           "return_traj": True,
                           "termination": "growth",
                          }, daemon=True)
uprocess.start()

/home/semyon/PycharmProjects/DiplomaPython/data/may_complex/1dfj.pdb


@> 4416 atoms and 1 coordinate set(s) were parsed in 0.05s.


write PDB(prody): 0.0347 sec
read PDB(openmm): 0.31346821784973145 sec
add hydrogens and extra particles(openmm): 18.66754961013794 sec
write PDB(openmm): 0.10308432579040527 sec


@> 8726 atoms and 1 coordinate set(s) were parsed in 0.07s.


1856 6870
construction of a system: 1.430229902267456 sec


@> Hessian was built in 7.60s.
@> 10 modes were calculated in 14.34s.


INIT ENERGY: -30170.72324883641
rmsd init: 0.3
RANDOM ENERGY: -4262.350630371911


@> Hessian was built in 7.62s.
@> 10 modes were calculated in 14.29s.



(main) CYCLE START

	(line) energy: 14813.024181660716
	(line) energy: 3601.7601291527826
	(line) energy: -4486.536335985606
	(line) energy: -10220.3917981376
	(line) energy: -14196.751978691638
	(line) energy: -16864.749636297325
	(line) energy: -18553.19396886926
	(line) energy: -19502.820880761064
	(line) energy: -19892.13892709113
	(line) energy: -19858.320577986597
	(line) energy: -19518.534205922544
	(line) energy: -18974.415594641694
(main) optimum:
       index: 9
       energy: -19892.13892709113
       force: 5670.999875768754
       upper_bound: 0.11343807013483424

(main) CYCLE START

	(line) energy: -13938.855489357637
	(line) energy: -15347.425521372446
	(line) energy: -16458.872640936148
	(line) energy: -17327.12806017592
	(line) energy: -18001.47565774252
	(line) energy: -18523.24603125399
	(line) energy: -18925.59842213249
	(line) energy: -19234.60770599141
	(line) energy: -19470.66110954483
	(line) energy: -19649.67394208115
	(line) energy: -19784.098233482066
	(line

       force: 2325.672232503767
       upper_bound: 0.006953209736092344

(main) CYCLE START



In [10]:
# terminate the process
uprocess.terminate()

In [9]:
# connect to descent output
%matplotlib
fig, ax = plt.subplots(2, 2, figsize=(10, 10))

animation_on = True

def animate(i):
    global full_list, objects
    if not animation_on:
        return
    try:
        obj = obj_q.get(False)
        if "energies" in obj:
            full_list.append(obj)
            ax[0, 0].clear()
            ax[0, 1].clear()
            ax[1, 0].clear()
            ax[0, 0].set_title("Energy (main)")
            ax[0, 1].set_title("RMSD (main)")
            ax[1, 0].set_title("Force (main)")
            n = len(obj["energies"])
            m = len(obj["forces"])
            k = len(obj["states"])
            if len(obj["energies"]) > 0:
                ax[0, 0].scatter(np.arange(0, n, 1), obj["energies"])
                state_0 = obj["states"][0]
                weights = np.ones((len(state_0),))
                ax[0, 1].scatter(np.arange(0, k, 1), [rmsd(state_0, s, weights) for s in obj["states"]])
                forces = obj["forces"]
                ax[1, 0].scatter(np.arange(0, m - 1, 1),
                        [(np.dot(forces[u], forces[u - 1]) /
                          np.linalg.norm(forces[u]) /
                          np.linalg.norm(forces[u - 1])) for u in range(1, m)])
        elif "energies_line" in obj:
            ax[1, 1].clear()
            ax[1, 1].set_title("Energy (line)")
            n = len(obj["energies_line"])
            if n > 0:
                ax[1, 1].scatter(np.arange(0, n, 1), obj["energies_line"])
    except Empty:
        pass
anim = animation.FuncAnimation(fig, animate, 200)

Using matplotlib backend: TkAgg


In [ ]:
# pause the animation
animation_on = False

In [ ]:
# or resume the animation
animation_on = True

In [ ]:
# CA comparison
calpha_0 = drs._refine_prot_init.select("calpha").copy()
nmw2.set_position(np.zeros(10,))
calpha_1 = drs._refine_prot.select("calpha").copy()
nmw2.set_position(opt_pos)
calpha_2 = drs._refine_prot.select("calpha").copy()

nca = len(calpha_0)
print(nca)
weights1 = np.ones((nca,))

In [ ]:
print("01", rmsd(calpha_0.getCoords(), calpha_1.getCoords(), weights1))
print("02", rmsd(calpha_0.getCoords(), calpha_2.getCoords(), weights1))
print("12", rmsd(calpha_1.getCoords(), calpha_2.getCoords(), weights1))

## Save trajectories
 1) save all states
 
 2) read them with mdtraj
 
 3) save them as .xtc
 
 4) enjoy your trajectories in VMD!!!

In [11]:
drs = DRSystem(str(project.output_path / "tmp_system.pdb"), 'amber14/protein.ff14SB.xml', refine="chain A", static="chain B")

@> 8726 atoms and 1 coordinate set(s) were parsed in 0.07s.


1856 6870


In [12]:
full_list = full_list[27:]

In [13]:
n_states = len(full_list)

trj = None
for i in range(n_states):
    drs.set_position(full_list[i]["states"][i])
    with open(project.output_path / "inter_pdb.pdb", "w") as input_file:
        drs._omm_protein.writeFile(positions=drs._omm_protein.positions,
                                   topology=drs._omm_protein.topology,
                                   file=input_file)
    if trj is not None:
        trj = trj.join(mdt.load(str(project.output_path / "inter_pdb.pdb")))
    else:
        trj = mdt.load(str(project.output_path / "inter_pdb.pdb"))
    print(trj.xyz.shape)

(1, 8726, 3)
(2, 8726, 3)
(3, 8726, 3)
(4, 8726, 3)
(5, 8726, 3)
(6, 8726, 3)
(7, 8726, 3)
(8, 8726, 3)
(9, 8726, 3)
(10, 8726, 3)
(11, 8726, 3)
(12, 8726, 3)
(13, 8726, 3)
(14, 8726, 3)
(15, 8726, 3)
(16, 8726, 3)
(17, 8726, 3)
(18, 8726, 3)
(19, 8726, 3)
(20, 8726, 3)
(21, 8726, 3)
(22, 8726, 3)
(23, 8726, 3)
(24, 8726, 3)
(25, 8726, 3)
(26, 8726, 3)
(27, 8726, 3)
(28, 8726, 3)
(29, 8726, 3)
(30, 8726, 3)
(31, 8726, 3)
(32, 8726, 3)
(33, 8726, 3)
(34, 8726, 3)
(35, 8726, 3)
(36, 8726, 3)
(37, 8726, 3)
(38, 8726, 3)
(39, 8726, 3)
(40, 8726, 3)
(41, 8726, 3)
(42, 8726, 3)
(43, 8726, 3)
(44, 8726, 3)
(45, 8726, 3)
(46, 8726, 3)
(47, 8726, 3)
(48, 8726, 3)
(49, 8726, 3)
(50, 8726, 3)
(51, 8726, 3)
(52, 8726, 3)
(53, 8726, 3)
(54, 8726, 3)
(55, 8726, 3)
(56, 8726, 3)
(57, 8726, 3)
(58, 8726, 3)
(59, 8726, 3)
(60, 8726, 3)
(61, 8726, 3)
(62, 8726, 3)
(63, 8726, 3)
(64, 8726, 3)
(65, 8726, 3)
(66, 8726, 3)
(67, 8726, 3)
(68, 8726, 3)
(69, 8726, 3)
(70, 8726, 3)
(71, 8726, 3)
(72, 8726, 3)
(

In [14]:
trj.save_pdb(str(project.output_path / "trajectory.pdb"))

## WARNING: it is an abandoned code down there.

In [ ]:
# from time to time
t0 = time.time()
for i in range(1):
    nmw1.get_energy()
print("Energy computation takes:", (time.time() - t0) / 10, "sec")
t0 = time.time()
for i in range(10):
    state = nmw1._system._simulation.context.getState(getEnergy=True)
    state.getPotentialEnergy().value_in_unit(kilojoule_per_mole)
print("Pure energy computation takes:", (time.time() - t0) / 10, "sec")
t0 = time.time()
for i in range(10):
    nmw1.get_force()
print("Force computation takes:", (time.time() - t0) / 10, "sec")
t0 = time.time()
for i in range(10):
    nmw1.set_position(np.random.normal(0.0, 2, 10))
print("NM space position update takes:", (time.time() - t0) / 10, "sec")

In [92]:
x = np.arange(2, 17.1, 1)
y0 = np.array([3, 8, 11, 13, 18, 26, 39, 54, 67, 79, 102, 127, 147, 172, 200, 232])
y1 = np.array([3, 4, 8, 14, 28, 48, 58, 77, 97, 111, 128, 165, 210, 252, 292, 332])
y2 = np.array([4, 13, 25, 51, 95, 148, 213, 292, 411, 520, 678, 790, 945, 1056, 1178, 1287])

In [93]:
%matplotlib
fig, ax = plt.subplots()
ax.scatter(x, y0 ** (1/3), color="grey")
ax.scatter(x, y1 ** (1/3), color="orange")
ax.scatter(x, y2 ** (1/3), color="green")

Using matplotlib backend: TkAgg


In [39]:
from scipy.optimize import least_squares
res0 = least_squares(lambda a: (y0 - a[0] * x ** 3) ** 2, np.ones((1,)))
res1 = least_squares(lambda a: (y1 - a[0] * x ** 3) ** 2, np.ones((1,)))
res2 = least_squares(lambda a: (y2 - a[0] * x ** 3) ** 2, np.ones((1,)))

In [66]:
a0 = res0.x[0]
a1 = res1.x[0]
a2 = res2.x[0]
print(a0, a1, a2)
print(a0 / a2, a1 / a2)
print(a2 / a0, a2 / a1)

0.05069907949156018 0.07196913802648248 0.2967683292347536
0.1708372305841825 0.24250949625272347
5.853524999091084 4.123549862797464


In [146]:
theta = np.arccos(1 - 2 * a0 / a2)
omega = a0 / a2 * np.pi * 4
n = a2 * 3 / 4 / np.pi
print("theta:", theta)
print("omega:", omega)
print("n:", n)

theta: 0.8522042436577324
omega: 2.1468039542515727
n: 0.0708482198262511


In [147]:
fpo = (1 - cos(2 * theta)) / 2 / (theta - np.sin(2 * theta) / 2)
print(fpo)

1.588645869600043


In [148]:
fsp = 4 / (1 - cos(2 * theta))
print(fsp)

3.529780409228429


In [149]:
fso = fsp * fpo
print(fso)

5.607571067715893


In [125]:
x = np.arange(0.1, theta + 0.01, 0.01)
f = (1 - np.cos(2 * x)) / 2 / (x - np.sin(2 * x) / 2)
plt.plot(x, f)